In [9]:
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json?api_key=AUZWyThfl5cObDMRt97J8zMk12LPMS3Fww6OCvOb"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{"generated_at":"2025-04-21T11:20:28+00:00","complexes":[{"id":"sr:complex:705","name":"Nacional","venues":[{"id":"sr:venue:70045","name":"Cancha Central","city_name":"Santiago","country_name":"Chile","country_code":"CHL","timezone":"America\/Santiago"}]},{"id":"sr:complex:1078","name":"Estadio de la Cartuja","venues":[{"id":"sr:venue:74856","name":"Centre Court","city_name":"Seville","country_name":"Spain","country_code":"ESP","timezone":"Europe\/Madrid"},{"id":"sr:venue:74858","name":"Court One","city_name":"Seville","country_name":"Spain","country_code":"ESP","timezone":"Europe\/Madrid"}]},{"id":"sr:complex:1495","name":"Sibur Arena","venues":[{"id":"sr:venue:1496","name":"COURT 1","city_name":"Saint Petersburg","country_name":"Russia","country_code":"RUS","timezone":"Europe\/Moscow"},{"id":"sr:venue:1500","name":"CENTER COURT","city_name":"Saint Petersburg","country_name":"Russia","country_code":"RUS","timezone":"Europe\/Moscow"},{"id":"sr:venue:62149","name":"Sibur Arena","city_na

In [11]:
import json
data= json.loads(response.text)
data

{'generated_at': '2025-04-21T11:20:28+00:00',
 'complexes': [{'id': 'sr:complex:705',
   'name': 'Nacional',
   'venues': [{'id': 'sr:venue:70045',
     'name': 'Cancha Central',
     'city_name': 'Santiago',
     'country_name': 'Chile',
     'country_code': 'CHL',
     'timezone': 'America/Santiago'}]},
  {'id': 'sr:complex:1078',
   'name': 'Estadio de la Cartuja',
   'venues': [{'id': 'sr:venue:74856',
     'name': 'Centre Court',
     'city_name': 'Seville',
     'country_name': 'Spain',
     'country_code': 'ESP',
     'timezone': 'Europe/Madrid'},
    {'id': 'sr:venue:74858',
     'name': 'Court One',
     'city_name': 'Seville',
     'country_name': 'Spain',
     'country_code': 'ESP',
     'timezone': 'Europe/Madrid'}]},
  {'id': 'sr:complex:1495',
   'name': 'Sibur Arena',
   'venues': [{'id': 'sr:venue:1496',
     'name': 'COURT 1',
     'city_name': 'Saint Petersburg',
     'country_name': 'Russia',
     'country_code': 'RUS',
     'timezone': 'Europe/Moscow'},
    {'id': '

In [13]:
complexes=data['complexes']
complexes_list=[]
for comp in complexes:
   complexes_list.append({
         "complex_id":comp["id"],
        "complex_name":comp["name"]
    })
    

In [15]:
import pandas as pd
df3=pd.DataFrame(complexes_list)
df3.head()

,complex_id,complex_name
0,sr:complex:705,Nacional
1,sr:complex:1078,Estadio de la Cartuja
2,sr:complex:1495,Sibur Arena
3,sr:complex:2375,Complexo de Tenis do Jamor
4,sr:complex:4032,Shree Shiv Chhatrapati Sports Complex


In [17]:
data=json.loads(response.text)
complexes=data["complexes"]

In [47]:
venues_list=[]
for comp in complexes:
    for venue in comp.get("venues", []):  
        venues_list.append({
            "venues_id": venue["id"],
            "name": venue["name"],
            "city_name": venue["city_name"],
            "country_name": venue["country_name"],
            "country_code": venue["country_code"],
            "timezone": venue["timezone"],
        })

In [49]:
import pandas as pd
df4=pd.DataFrame(venues_list)
df4.head()

,venues_id,name,city_name,country_name,country_code,timezone
0,sr:venue:70045,Cancha Central,Santiago,Chile,CHL,America/Santiago
1,sr:venue:74856,Centre Court,Seville,Spain,ESP,Europe/Madrid
2,sr:venue:74858,Court One,Seville,Spain,ESP,Europe/Madrid
3,sr:venue:1496,COURT 1,Saint Petersburg,Russia,RUS,Europe/Moscow
4,sr:venue:1500,CENTER COURT,Saint Petersburg,Russia,RUS,Europe/Moscow


In [51]:
venues_list=[]
for comp in complexes:
    venues = comp.get('venues')
    if venues and len(venues) > 0:
        venue = venues[0]
        venues_list.append({
            "venue_id": venue["id"],
            "venue_name": venue["name"],
            "city_name": venue["city_name"],
            "country_name": venue["country_name"],
            "country_code": venue["country_code"],
            "timezone": venue["timezone"],
            "complex_id": comp["id"]
        })

In [55]:
import pandas as pd
df4=pd.DataFrame(venues_list)
df4.head()


,venue_id,venue_name,city_name,country_name,country_code,timezone,complex_id
0,sr:venue:70045,Cancha Central,Santiago,Chile,CHL,America/Santiago,sr:complex:705
1,sr:venue:74856,Centre Court,Seville,Spain,ESP,Europe/Madrid,sr:complex:1078
2,sr:venue:1496,COURT 1,Saint Petersburg,Russia,RUS,Europe/Moscow,sr:complex:1495
3,sr:venue:57912,Central,Oeiras,Portugal,PRT,Europe/Lisbon,sr:complex:2375
4,sr:venue:13912,Centre Court,Ostrava,Czechia,CZE,Europe/Prague,sr:complex:5526


In [27]:
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    # port=3306
)

mycursor = mydb.cursor(buffered=True)
mycursor.execute("use project")
mycursor.execute("create table Complexes(complex_id varchar(50) Primary key,complex_name varchar(100) Not null)")

In [29]:
insert_query = "INSERT IGNORE INTO Complexes (complex_id, complex_name) VALUES (%s, %s)"
for index, row in df3.iterrows():
    mycursor.execute(insert_query, (row['complex_id'], row['complex_name']))
mydb.commit()

In [31]:
mycursor.execute("""
CREATE TABLE Venues (
    venue_id VARCHAR(50) PRIMARY KEY,
    city_name VARCHAR(100) NOT NULL,
    venue_name VARCHAR(100) NOT NULL,
    country_name VARCHAR(100)NOT NULL,
    country_code CHAR(3) NOT NULL,
    timezone VARCHAR(100) NOT NULL,
    complex_id VARCHAR(50),
    FOREIGN KEY (complex_id) REFERENCES Complexes(complex_id)
)
""")

In [33]:

insert_query = "INSERT IGNORE INTO Venues (venue_id, venue_name, city_name, country_name, country_code, timezone, complex_id) VALUES (%s, %s, %s, %s, %s, %s, %s)"
for index, row in df4.iterrows():
    mycursor.execute(insert_query, (
        row['venue_id'],
        row['venue_name'],
        row['city_name'],
        row['country_name'],
        row['country_code'],
        row['timezone'],
        row['complex_id']
    ))
mydb.commit()

In [37]:
#1.List all venues along with their associated complex name
from tabulate import tabulate
mycursor.execute("""
    SELECT 
        c.complex_name,
        v.venue_name
    FROM 
        complexes c
    JOIN 
        venues v ON c.complex_id = v.complex_id
""")

out = mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='grid'))

+------------------------------------------------------------+------------------------------------------------------------+
| complex_name                                               | venue_name                                                 |
+============================================================+============================================================+
| Shenzhen International Tennis Center                       | Stadium                                                    |
+------------------------------------------------------------+------------------------------------------------------------+
| Shenzhen Longgang Sports Center                            | Court 2                                                    |
+------------------------------------------------------------+------------------------------------------------------------+
| Complexe Tennistique Marie-Louise Lhuillier                | Stade Numa-Daly Magenta                                    |
+-------

In [39]:
#2.Count the number of venues in each complex
mycursor.execute("""
    SELECT 
        c.complex_id, 
        c.complex_name, 
        COUNT(v.venue_id) AS venue_count
    FROM 
        Complexes c
    JOIN 
        Venues v ON c.complex_id = v.complex_id
    GROUP BY 
        c.complex_name; # c.complex_id, 
""")
out = mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='grid'))


+------------------+------------------------------------------------------------+---------------+
| complex_id       | complex_name                                               |   venue_count |
+==================+============================================================+===============+
| sr:complex:65717 | A.S.D. Tennis Club Como                                    |             1 |
+------------------+------------------------------------------------------------+---------------+
| sr:complex:39866 | AAJB Tennis                                                |             1 |
+------------------+------------------------------------------------------------+---------------+
| sr:complex:61026 | Abama Tennis Academy                                       |             1 |
+------------------+------------------------------------------------------------+---------------+
| sr:complex:73463 | Academy Zhangjiagang                                       |             1 |
+------------------+

In [63]:
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    # port=3306
)

mycursor = mydb.cursor(buffered=True)
mycursor.execute("use project")


In [65]:
#3.Get details of venues in a specific country (e.g., Chile)
mycursor.execute("SELECT * FROM venues where country_name = 'chile'")
out = mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='grid'))

+----------------+--------------+--------------------+----------------+----------------+------------------+------------------+
| venue_id       | city_name    | venue_name         | country_name   | country_code   | timezone         | complex_id       |
+================+==============+====================+================+================+==================+==================+
| sr:venue:1602  | Santiago     | Cancha Central     | Chile          | CHL            | America/Santiago | sr:complex:62419 |
+----------------+--------------+--------------------+----------------+----------------+------------------+------------------+
| sr:venue:17100 | Santiago     | Court 1            | Chile          | CHL            | America/Santiago | sr:complex:50605 |
+----------------+--------------+--------------------+----------------+----------------+------------------+------------------+
| sr:venue:26202 | Antofagasta  | Cancha 7           | Chile          | CHL            | America/Santiago | sr:

In [67]:
#4.Identify all venues and their timezones
mycursor.execute("SELECT venue_id, venue_name, timezone FROM Venues")
out = mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='grid'))

+----------------+------------------------------------------------------------+--------------------------------+
| venue_id       | venue_name                                                 | timezone                       |
+================+============================================================+================================+
| sr:venue:10390 | Stadium                                                    | Asia/Shanghai                  |
+----------------+------------------------------------------------------------+--------------------------------+
| sr:venue:10394 | Court 2                                                    | Asia/Shanghai                  |
+----------------+------------------------------------------------------------+--------------------------------+
| sr:venue:10644 | Stade Numa-Daly Magenta                                    | Pacific/Noumea                 |
+----------------+------------------------------------------------------------+-----------------

In [69]:
#5.Find complexes that have more than one venue
mycursor.execute("""
    SELECT 
        c.complex_id, 
        c.complex_name, 
        COUNT(v.venue_id) AS venue_count
    FROM 
        Complexes c
    JOIN 
        Venues v ON c.complex_id = v.complex_id
    GROUP BY 
        c.complex_name
    HAVING 
        venue_count > 1;
""")

print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='grid'))

+----------------+------------------------------------------------------------+--------------------------------+
| complex_id     | complex_name                                               | venue_count                    |
+================+============================================================+================================+
| sr:venue:10390 | Stadium                                                    | Asia/Shanghai                  |
+----------------+------------------------------------------------------------+--------------------------------+
| sr:venue:10394 | Court 2                                                    | Asia/Shanghai                  |
+----------------+------------------------------------------------------------+--------------------------------+
| sr:venue:10644 | Stade Numa-Daly Magenta                                    | Pacific/Noumea                 |
+----------------+------------------------------------------------------------+-----------------

In [71]:
#6.List venues grouped by country
mycursor.execute("SELECT venue_id, venue_name,country_name FROM Venues Group by country_name")
out = mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='grid'))

+----------------+-------------------------+----------------------+
| venue_id       | venue_name              | country_name         |
+================+=========================+======================+
| sr:venue:15236 | Cancha Central          | Argentina            |
+----------------+-------------------------+----------------------+
| sr:venue:1535  | Pat Rafter Arena        | Australia            |
+----------------+-------------------------+----------------------+
| sr:venue:1385  | Centre Court            | Austria              |
+----------------+-------------------------+----------------------+
| sr:venue:28179 | Court 3                 | Bahrain              |
+----------------+-------------------------+----------------------+
| sr:venue:15640 | Court 1                 | Belgium              |
+----------------+-------------------------+----------------------+
| sr:venue:23922 | Cancha 1                | Bolivia              |
+----------------+-------------------------+----

In [238]:
#7.Find all venues for a specific complex (e.g., Nacional)
mycursor.execute("""
    SELECT  
        c.complex_name, 
        v.venue_name
    FROM 
        Complexes c
    JOIN 
        Venues v ON c.complex_id = v.complex_id
    WHERE
    c.complex_name = 'Nacional'
""")
out = mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description], tablefmt='grid'))

+----------------+----------------+
| complex_name   | venue_name     |
+================+================+
| Nacional       | Cancha Central |
+----------------+----------------+


In [73]:
mycursor.execute("select * from complexes limit 10")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='grid'))

+------------------+------------------------------+
| complex_id       | complex_name                 |
+==================+==============================+
| sr:complex:10063 | Kindarena                    |
+------------------+------------------------------+
| sr:complex:1078  | Estadio de la Cartuja        |
+------------------+------------------------------+
| sr:complex:11769 | Estadio German Becker        |
+------------------+------------------------------+
| sr:complex:12241 | Palais des sports de Gerland |
+------------------+------------------------------+
| sr:complex:1495  | Sibur Arena                  |
+------------------+------------------------------+
| sr:complex:15014 | Palais Des Sport             |
+------------------+------------------------------+
| sr:complex:15371 | Palasport                    |
+------------------+------------------------------+
| sr:complex:16020 | Salle Steredenn              |
+------------------+------------------------------+
| sr:complex

In [228]:
from tabulate import tabulate
mycursor.execute("select * from venues limit 10")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='grid'))

+----------------+--------------+-------------------------+----------------+----------------+-------------------+------------------+
| venue_id       | city_name    | venue_name              | country_name   | country_code   | timezone          | complex_id       |
+================+==============+=========================+================+================+===================+==================+
| sr:venue:10390 | Shenzhen     | Stadium                 | China          | CHN            | Asia/Shanghai     | sr:complex:38603 |
+----------------+--------------+-------------------------+----------------+----------------+-------------------+------------------+
| sr:venue:10394 | Shenzhen     | Court 2                 | China          | CHN            | Asia/Shanghai     | sr:complex:33334 |
+----------------+--------------+-------------------------+----------------+----------------+-------------------+------------------+
| sr:venue:10644 | Noumea       | Stade Numa-Daly Magenta | New Caled

In [26]:
from tabulate import tabulate
mycursor.execute("SELECT DISTINCT country_name FROM venues")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='grid'))

+----------------------+
| country_name         |
+======================+
| China                |
+----------------------+
| New Caledonia        |
+----------------------+
| Japan                |
+----------------------+
| Morocco              |
+----------------------+
| Portugal             |
+----------------------+
| Germany              |
+----------------------+
| France               |
+----------------------+
| USA                  |
+----------------------+
| Romania              |
+----------------------+
| Uzbekistan           |
+----------------------+
| Republic of Korea    |
+----------------------+
| Brazil               |
+----------------------+
| Thailand             |
+----------------------+
| Kazakhstan           |
+----------------------+
| Mexico               |
+----------------------+
| Austria              |
+----------------------+
| Sweden               |
+----------------------+
| Russia               |
+----------------------+
| Tunisia              |
